In [78]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

#读取数据
train_df = pd.read_csv("../data_ai/toUser/train.csv")
# A
# test_df = pd.read_csv("../data_ai/toUser/testA.csv")
# sub = pd.read_csv("../data_ai/toUser/submitA.csv")
# B
test_df = pd.read_csv("../data_ai/testB.csv")
sub = pd.read_csv("../data_ai/toUser/submitA.csv")

#合并训练集，测试集
train_df['is_test'] = 0
test_df['is_test'] = 1
df_all = pd.concat([train_df,test_df],axis=0,ignore_index=True)

In [ ]:
#时间特征
def extract_date_features(data,date_col):
    
    data[date_col]      = pd.to_datetime(data[date_col])
    
    data['Year']             = data[date_col].dt.year
    data['Month']            = data[date_col].dt.month
    data['Day']              = data[date_col].dt.day
    data['Weekday']          = data[date_col].dt.weekday
    data['Day_of_year']      = data[date_col].dt.dayofyear
    data['Week_of_year']     = data[date_col].dt.isocalendar().week.astype(int)
    data['Quarter']          = data[date_col].dt.quarter
    data['Is_month_start']   = data[date_col].dt.is_month_start.astype(int)
    data['Is_month_end']     = data[date_col].dt.is_month_end.astype(int)
    data['Is_quarter_start'] = data[date_col].dt.is_quarter_start.astype(int)
    data['Is_quarter_end']   = data[date_col].dt.is_quarter_end.astype(int)
    data['Is_year_start']    = data[date_col].dt.is_year_start.astype(int)
    data['Is_year_end']      = data[date_col].dt.is_year_end.astype(int)
    data['Days_in_month']    = data[date_col].dt.days_in_month.astype(int)
    data['Is_leap_year']     = data[date_col].dt.is_leap_year.astype(int)
    data['Is_weekend']       = data['Weekday'].apply(lambda x: x >= 5).astype(int)
    data['Days_till_month_end']    = data['Days_in_month'] - data['Day']
    data['Days_since_month_start'] = data['Day'] - 1
    data['Week_of_month']          = (data['Day'] - 1) // 7 + 1
    data['Weekday_of_month']       = (data['Day'] - 1) % 7 + 1
    
    return data

df_all = extract_date_features(df_all,'join_date')

In [80]:
#缺失比例大于50%的列,是否缺失二分类
for col in ['sl_flag','sl_type']:
    df_all[col] = np.where(df_all[col].isna(),1,0)
   
#缩放
ll_clos = ['ztc_gprs_res','avg3_tc_ll','avg3_tw_ll','avg3_dou','avg3_sl_ll',
           'sl_ll2','avg3_video_app_ll','avg3_music_app_ll','avg3_game_app_ll']
for col in ll_clos:
    df_all[col] = df_all[col]/1024
    
#编码
df_all['zfk_type'] = df_all['zfk_type'].factorize()[0]
df_all['jt_5gwl_flag'] = df_all['jt_5gwl_flag'].factorize()[0]
df_all['term_brand'] = df_all['term_brand'].factorize()[0]
df_all['area_code'] = df_all['area_code'].astype('str').factorize()[0]

#替换异常值
df_all['age'] = df_all['age'].replace(999,np.nan)
df_all['term_brand'] = df_all['term_brand'].replace(-1,np.nan)

# # #用户特征
# #交叉统计特征
#area_code(81.4856727064221)
df_all['area_code_change_equip_period_avg_mean'] = df_all['area_code'].map(df_all.groupby('area_code')['change_equip_period_avg'].mean())
df_all['area_code_term_price_mean'] = df_all['area_code'].map(df_all.groupby('area_code')['term_price'].mean())
df_all['area_code_ztc_gprs_res_mean'] = df_all['area_code'].map(df_all.groupby('area_code')['ztc_gprs_res'].mean())
# #age(81.67717928649066)
df_all['age_change_equip_period_avg_mean'] = df_all['age'].map(df_all.groupby('age')['change_equip_period_avg'].mean())
df_all['age_term_price_mean'] = df_all['age'].map(df_all.groupby('age')['term_price'].mean())
df_all['age_ztc_gprs_res_mean'] = df_all['age'].map(df_all.groupby('age')['ztc_gprs_res'].mean())
#user_type、group_type
df_all['enc_feat1'] = (df_all['user_type'].astype('str') + '_' + df_all['group_type'].astype('str')).factorize()[0]
#jt_5gzd_flag、jt_5gwl_flag
df_all['enc_feat2'] = (df_all['jt_5gzd_flag'].astype('str') + '_' + df_all['jt_5gwl_flag'].astype('str')).factorize()[0]
#term_brand
df_all['term_brand_term_price_mean'] = df_all['term_brand'].map(df_all.groupby('term_brand')['term_price'].mean())
df_all['term_brand_change_equip_period_avg_mean'] = df_all['term_brand'].map(df_all.groupby('term_brand')['change_equip_period_avg'].mean())
df_all['term_brand_ztc_gprs_res_mean'] = df_all['term_brand'].map(df_all.groupby('term_brand')['ztc_gprs_res'].mean())
df_all['term_brand_ztc_price_mean'] = df_all['term_brand'].map(df_all.groupby('term_brand')['ztc_price'].mean())
                                                 
# #套餐资费-内容洞察特征(81.5552985500075)
# #组合特征
#流量占比
df_all['avg3_video_app_ll_sl_ll2_sub'] = df_all['avg3_video_app_ll']/df_all['sl_ll2']
df_all['avg3_music_app_ll_sl_ll2_sub'] = df_all['avg3_music_app_ll']/df_all['sl_ll2']
df_all['avg3_game_app_ll_sl_ll2_sub'] = df_all['avg3_game_app_ll']/df_all['sl_ll2']
#流量费用占比
df_all['avg3_tc_price_avg3_tot_fee_sub'] = df_all['avg3_tc_price']/df_all['avg3_tot_fee']
df_all['avg3_ctll_fee_avg3_tot_fee_sub'] = df_all['avg3_ctll_fee']/df_all['avg3_tot_fee']
df_all['avg3_ctyy_fee_avg3_tot_fee_sub'] = df_all['avg3_ctll_fee']/df_all['avg3_ctyy_fee']

# #不同数字产品占比情况(81.58818559286505)
df_all['video_app1_cnt_rate'] = df_all['avg3_video_app1_cnt']/df_all[['avg3_video_app1_cnt','avg3_music_app1_cnt','avg3_game_app1_cnt']].sum(axis=1)
df_all['video_app2_cnt_rate'] = df_all['avg3_video_app2_cnt']/df_all[['avg3_video_app2_cnt','avg3_music_app2_cnt','avg3_game_app2_cnt']].sum(axis=1)
df_all['video_app_ll_rate'] = df_all['avg3_video_app_ll']/df_all[['avg3_video_app_ll','avg3_music_app_ll','avg3_game_app_ll']].sum(axis=1)

df_all['music_app1_cnt_rate'] = df_all['avg3_music_app1_cnt']/df_all[['avg3_video_app1_cnt','avg3_music_app1_cnt','avg3_game_app1_cnt']].sum(axis=1)
df_all['music_app2_cnt_rate'] = df_all['avg3_music_app2_cnt']/df_all[['avg3_video_app2_cnt','avg3_music_app2_cnt','avg3_game_app2_cnt']].sum(axis=1)
df_all['music_app_ll_rate'] = df_all['avg3_music_app_ll']/df_all[['avg3_video_app_ll','avg3_music_app_ll','avg3_game_app_ll']].sum(axis=1)

df_all['game_app1_cnt_rate'] = df_all['avg3_game_app1_cnt']/df_all[['avg3_video_app1_cnt','avg3_music_app1_cnt','avg3_game_app1_cnt']].sum(axis=1)
df_all['game_app2_cnt_rate'] = df_all['avg3_game_app2_cnt']/df_all[['avg3_video_app2_cnt','avg3_music_app2_cnt','avg3_game_app2_cnt']].sum(axis=1)
df_all['game_app_ll_rate'] = df_all['avg3_game_app_ll']/df_all[['avg3_video_app_ll','avg3_music_app_ll','avg3_game_app_ll']].sum(axis=1)
#数字产品使用情况
df_all['app1_cnt_sum'] = df_all[['avg3_video_app1_cnt','avg3_music_app1_cnt','avg3_game_app1_cnt']].sum(axis=1)
df_all['app1_cnt_mean'] = df_all[['avg3_video_app1_cnt','avg3_music_app1_cnt','avg3_game_app1_cnt']].mean(axis=1)
df_all['app2_cnt_mean'] = df_all[['avg3_video_app2_cnt','avg3_music_app2_cnt','avg3_game_app2_cnt']].mean(axis=1)
df_all['app_ll_mean'] = df_all[['avg3_video_app2_cnt','avg3_music_app_ll','avg3_game_app_ll']].mean(axis=1)

#交叉统计特征(81.62299066727917)
#APP使用个数
for col_i in ['area_code','age']:
    for col_j in ['avg3_video_app1_cnt','avg3_music_app1_cnt','avg3_game_app1_cnt']:
        df_all[f'{col_i}_{col_j}_mean'] = df_all[col_i].map(df_all.groupby(col_i)[col_j].mean())
    
#APP使用天次数
for col_i in ['area_code','age',]:
    for col_j in ['avg3_video_app2_cnt','avg3_music_app2_cnt','avg3_game_app2_cnt']:
        df_all[f'{col_i}_{col_j}_mean'] = df_all[col_i].map(df_all.groupby(col_i)[col_j].mean())
    
#APP使用总流量
for col_i in ['area_code','age',]:
    for col_j in ['avg3_video_app_ll','avg3_music_app_ll','avg3_game_app_ll']:
        df_all[f'{col_i}_{col_j}_mean'] = df_all[col_i].map(df_all.groupby(col_i)[col_j].mean())
        
#业务使用
#'avg3_dou','avg3_mou','avg3_llct_cnt','avg3_ll_bhd','avg3_sl_ll','avg3_tc_price','avg3_tot_fee','avg3_ctll_fee','avg3_ctyy_fee'
for col_i in ['area_code','age',]:
    for col_j in ['avg3_ll_bhd','avg3_tc_price','ll_bhd','avg3_dou','avg3_mou','avg3_llct_cnt',
                  'avg3_ll_bhd','avg3_sl_ll','avg3_tc_price','avg3_tot_fee','avg3_ctll_fee','avg3_ctyy_fee']:
        df_all[f'{col_i}_{col_j}_mean'] = df_all[col_i].map(df_all.groupby(col_i)[col_j].mean())

df_all['avg3_tc_ll_avg3_tw_ll_sub'] = df_all['avg3_tc_ll']/df_all['avg3_tw_ll']
df_all['avg3_tc_ll_avg3_tw_ll_sum'] = df_all['avg3_tc_ll']+df_all['avg3_tw_ll']

In [ ]:
#删除方差为0
drops_cols = [col for col in df_all.drop(columns=['user_id','join_date']) if df_all[col].var()==0]

#类别特征
cat_cols = ['area_code','gender_id','zfk_type','group_type','jt_5gzd_flag','jt_5gwl_flag',
            'avg3_llb_flag','avg3_llct_cnt','avg3_yyct_cnt']

#分离训练集，测试集
df_all = df_all.replace([-np.inf,np.inf],0)
df_all = df_all.fillna(0)
train_df = df_all[df_all['is_test']==0]
test_df = df_all[df_all['is_test']==1]
train_df['sample_flag'] = train_df['sample_flag']-1

#重复值删除
train_df = train_df.drop_duplicates(subset=[col for col in test_df if col not in ['user_id','join_date','sample_flag','is_test']],
                                    keep='first', inplace=False).reset_index(drop=True)
feats = [col for col in test_df if col not in ['user_id','join_date','sample_flag','is_test']+drops_cols]                                                                                                                                                                                      
len(feats)

In [ ]:
#内存压缩
import time
def reduce_mem_usage(df):
    starttime = time.time()
    numerics = ['int16','int32','int64','float16','float32 ','float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[ col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[ col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int' :
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8 ).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64 ).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype( np.float64)
    end_mem = df.memory_usage( ).sum() / 1024**2
    print('Start Memory usage is: {:.2f} MB'.format(start_mem))
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

reduce_mem_usage(train_df)
reduce_mem_usage(test_df)

In [ ]:
#lgb
import warnings
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.utils.class_weight import compute_sample_weight as cse
from sklearn.metrics import roc_auc_score,accuracy_score,classification_report,f1_score,precision_score, recall_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 标准化
def scale(train_features,test_features):
    scaler=StandardScaler()
    scaler.fit(train_features)
    train_features=pd.DataFrame(scaler.transform(train_features),columns=test_features.keys())
    test_features=pd.DataFrame(scaler.transform(test_features),columns=test_features.keys())
    return train_features.values,test_features.values

# 自定义评估函数
def my_metrice(preds, eval_data):
    label = eval_data.get_label()
    preds = np.argmax(preds.reshape(3,-1), axis=0)
    prc = precision_score(label, preds, average='macro')
    rec = recall_score(label, preds, average='macro')
    score = (0.7*prc + 0.3*rec)*100
    return ('score', score, True) 

# 评估函数
def eva_func(true_labels,preds):
    prc = precision_score(true_labels, preds, average='macro')
    rec = recall_score(true_labels, preds, average='macro')
    score = (0.7*prc + 0.3*rec)*100
    return score
    
def lgb_model(train_x, train_y, test_x):
    seeds=[9999]
    oof = np.zeros([train_x.shape[0],3])
    test_predict = np.zeros([test_x.shape[0],3])
    feat_imp_df = pd.DataFrame()
    feat_imp_df['feature'] = train_x.columns
    feat_imp_df['imp'] = 0
    for seed in seeds:
        print('Seed:',seed)
        folds = 5
        kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
        scores = []
        train_x =train_x.values
        train_y = train_y.values
        #5折交叉验证
        for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
            print("|  Model  Fold  {}  Training Start           |".format(str(i + 1)))         
            trn_x, trn_y, val_x, val_y = train_x[train_index], train_y[train_index],\
                                         train_x[valid_index], train_y[valid_index] 
            # mdoel params
            params={"boosting_type": "gbdt","objective": "multiclass","metric": ["multi_error"],
                    'num_class':3,"verbose": -1,'learning_rate':0.02,'num_leaves':128,'max_bin':155,
                    'feature_fraction':0.8,'bagging_fraction':0.8
                     }

            dtrain = lgb.Dataset(trn_x, label=trn_y,categorical_feature=[])
            dval = lgb.Dataset(val_x, label=val_y,categorical_feature=[])
            # 模型定义
            model = lgb.train(params, dtrain, valid_sets=[dval], num_boost_round=20000,
                              callbacks=[lgb.early_stopping(50), lgb.log_evaluation(10)],
                              feval=my_metrice)
            # predict
            val_pred  = model.predict(val_x,num_iteration=model.best_iteration)
            test_pred = model.predict(test_x,num_iteration=model.best_iteration)
            # feature_importance
            feat_imp_df['imp'] += model.feature_importance(importance_type='gain') / folds/ len(seeds)
            feat_imp_df = feat_imp_df.sort_values(by='imp', ascending=False).reset_index(drop=True)
            feat_imp_df['rank'] = range(feat_imp_df.shape[0])
            # save
            oof[valid_index] = val_pred / len(seeds)
            test_predict += test_pred / kf.n_splits / len(seeds)
            
            # 模型评估
            val_pred = np.argmax(val_pred,axis=1)
            score = eva_func(val_y,val_pred)
            print('Fold {} score : {}'.format(i+1,score))
            scores.append(score)
            print("***AVG_score :{} std :{}***".format(np.mean(scores),np.std(scores)))
            print(                                                                    )
        print('oof:',classification_report(train_y,np.argmax(oof,axis=1)))
        print('oof_score:',eva_func(train_y,np.argmax(oof,axis=1)))
        
    return oof, test_predict,feat_imp_df

# 训练LGB模型
lgb_oof, lgb_test, lgb_imp_df = lgb_model(train_df[feats], train_df['sample_flag'], test_df[feats])

In [ ]:
#提交
sub['user_id'] = list(test_df['user_id'])
sub['predtype'] = np.argmax(lgb_test,axis=1)+1
sub.to_csv('result.csv',index = False)

In [ ]:
! castlecli --third honghu --token 66f8ee1c068eb479612599715244f9fa --source result.csv